In [1]:
from tqdm import tqdm
from typing import List, Dict
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation,PacketFlowRepressentation,TimeslotRepresentation
from flowprintOptimal.sekigo.dataAnalysis.vNATDataFrameProcessor import VNATDataFrameProcessor
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
import random
from flowprintOptimal.sekigo.flowUtils.flowDatasets import PacketFlowDataset, BaseFlowDataset, DDQNActivityDataset
from torch.utils.data import Dataset,DataLoader
from torchsampler import ImbalancedDatasetSampler
from sklearn.model_selection import train_test_split
from flowprintOptimal.sekigo.flowUtils.commons import normalizePacketRep
import os
from joblib import Parallel, delayed
from flowprintOptimal.sekigo.flowUtils.commons import saveFlows,loadFlows
from flowprintOptimal.sekigo.dataAnalysis.dataFrameProcessor import UTMobileNetProcessor
from flowprintOptimal.sekigo.flowUtils.dataGetter import getTrainTestOOD
from sklearn.metrics import confusion_matrix
import json
from flowprintOptimal.sekigo.modeling.trainers import NNClassificationTrainer
from flowprintOptimal.sekigo.modeling.neuralNetworks import LSTMNetwork,TransformerGenerator,CNNNetwork1D, LSTMDuelingNetwork
from flowprintOptimal.sekigo.modeling.loggers import Logger
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
from flowprintOptimal.sekigo.earlyClassification.DQL.core import MemoryElement,Rewarder,State,SumTree
from flowprintOptimal.sekigo.earlyClassification.DQL.pMemory import PMemory
from flowprintOptimal.sekigo.earlyClassification.DQL.memoryFiller import MemoryFiller
from flowprintOptimal.sekigo.earlyClassification.DQL.datasets import MemoryDataset
from flowprintOptimal.sekigo.earlyClassification.DQL.trainers import EarlyClassificationtrainer,PEarlyClassificationTrainer
from flowprintOptimal.sekigo.utils.documentor import Documenter
from flowprintOptimal.sekigo.utils.evaluations import Evaluator,EarlyEvaluation
import warnings
from copy import deepcopy
import torch.nn as nn



warnings.filterwarnings('ignore')

In [2]:
configs = dict(
    name = "UNIBS_no_sample_ood_dueling_P",
    description = "No Sample UTMobileNet dueling, reward function gives out 0 for ood with priority (.35,.4)",
    
    common_config = dict(
        max_timesteps = 15,
        min_timesteps = 8
    ),
    
    full_model_kwargs = dict(
        lstm_hidden_size = 256,
        layers= 2, lstm_input_size = 3
    ),

    early_model_kwargs = dict(
        lstm_input_size= 3,lstm_hidden_size= 256,layers = 2                    
    ),
    
    data_config = dict(
        dataset_name = "unibs",
        subsampleConfig = None,#dict(max_gap = 20, min_gap = 5),                             
        max_flow_length = 80, # in seconds  ( each flow sample cannot excede this length)
        test_size = .2,
        ood_classes = ["Skype"],
        do_balance = False,
        data_type = "packet_representation"

    ),

    rewarder_config = dict(
        l = .5
    ),

    dataset_config = dict(
        aug = [0,.2]
    ),

    memory_fillter_config = dict(
        ood_config = dict(ood_aug = [.6,.9], ood_prob = .2),
        min_length = 5,
        use_balancer = False
    ),
    full_trainer_config = dict(
        use_sampler = False
    ),
    early_trainer_config = dict(
        use_sampler = False  # this is for giving more weight to wait samples
    )

)

In [3]:
train_flows,test_flows,ood_flows = getTrainTestOOD(**configs["data_config"], max_timesteps= configs["common_config"]["max_timesteps"],
                                                   min_timesteps= configs["common_config"]["min_timesteps"]
                                                   )

full class distrubation
BROWSERS    42855
P2P         21514
OTHER        5218
MAIL         4521
Skype        1280
Name: count, dtype: int64
using no sampling
filtering max_flow_length = 80
post num packet filter class distrubation
BROWSERS    39999
P2P         20831
OTHER        4779
MAIL         4516
Skype        1056
Name: count, dtype: int64
------------------------------
train class distrubation
BROWSERS    31933
P2P         16668
OTHER        3873
MAIL         3626
Name: count, dtype: int64
test class distrubation
BROWSERS    8066
P2P         4163
OTHER        906
MAIL         890
Name: count, dtype: int64


In [4]:
if configs["data_config"]["data_type"] == "packet_representation":
    train_dataset = PacketFlowDataset(flows= train_flows,label_to_index= None,aug= configs["dataset_config"]["aug"])
    test_dataset = PacketFlowDataset(flows= test_flows,label_to_index= train_dataset.label_to_index)
    ood_dataset = PacketFlowDataset(flows= ood_flows, label_to_index= None) if (ood_flows != None and len(ood_flows) != 0) else None
else:
    train_dataset = DDQNActivityDataset(flows= train_flows,label_to_index= None, aug= configs["dataset_config"]["aug"])
    test_dataset = DDQNActivityDataset(flows= test_flows,label_to_index= train_dataset.label_to_index)
    ood_dataset = DDQNActivityDataset(flows= ood_flows, label_to_index= None) if (ood_flows != None and len(ood_flows) != 0) else None

    configs["early_model_kwargs"]["lstm_input_size"] = 7
    configs["full_model_kwargs"]["lstm_input_size"] = 7


num_labels = len(train_dataset.label_to_index)
configs["full_model_kwargs"]["output_dim"] = num_labels 
configs["early_model_kwargs"]["output_dim"] = num_labels + 1
configs["rewarder_config"]["num_labels"] = num_labels
configs["rewarder_config"]["max_length"] = configs["common_config"]["max_timesteps"]

In [5]:
#classifier_ = CNNNetwork1D( **configs["full_model_kwargs"])#LSTMNetwork(lstm_hidden_size= 64,lstm_input_size=6,output_dim = len(all_train_dataset.label_to_index))
classifier_ = LSTMNetwork(**configs["full_model_kwargs"])
logger= Logger(name= "classification",verbose= True)
logger.default_step_size = 500
classification_trainer = NNClassificationTrainer(classifier = classifier_,device= device,logger= logger)
#classification_trainer.train(train_dataset= train_dataset,test_dataset= test_dataset,epochs=500,batch_size= 64,lr= .0003,use_balanced_sampler= configs["full_trainer_config"]["use_sampler"])

In [6]:
class Rewarder:
    def __init__(self,max_length,l,num_labels : int):
        self.max_length = max_length
        self.l = l # l is smaller than 1
        self.num_labels = num_labels

    def reward(self,state : State,action : int):
        if state.label == action:
            # reward 1 on a correct prediction
            return 1, True
        else:
            # either incorrect or wait
            # wait 
            # treat the wait action with a negative reward
            if action == self.num_labels:
                if state.length == len(state.timeseries):
                    # it is the last timestamp
                    if state.label == -1:
                        return 0, True#-self.l*(state.length/self.max_length),True
                    else:
                        return -self.l/self.max_length, True
                else:
                    if state.label == -1:
                        return 0, False
                    else:
                        return  -self.l/self.max_length, False#-self.l*(state.length/self.max_length), False
            else:
                # incorrect
                return -1,True

In [7]:
rewarder = Rewarder(**configs["rewarder_config"])
memory_filler = MemoryFiller(dataset= train_dataset,rewarder= rewarder, min_length= configs["memory_fillter_config"]["min_length"],
                              max_length= rewarder.max_length,ood_config= configs["memory_fillter_config"]["ood_config"], use_balancer= configs["memory_fillter_config"]["use_balancer"]
                              )

In [8]:
memory = memory_filler.processDataset()
print(len(memory))

5668240


In [9]:
p_memory = PMemory(memories= memory, capacity= 500000,batch_size= 128)
predictor = LSTMDuelingNetwork(**configs["early_model_kwargs"])
logger = Logger(verbose= True)
logger.default_step_size = 1000

In [10]:
ddq_model = PEarlyClassificationTrainer(predictor= predictor,train_dataset = train_dataset,test_dataset= test_dataset,p_memory= p_memory,
                                       ood_dataset= ood_dataset,min_length= configs["memory_fillter_config"]["min_length"],
                                       logger= logger,device=device,model_replacement_steps= 500)

In [11]:
ddq_model.train(iterations= 500000, lr = 3e-4, lam = .99)

 ---- 1000 metric q_loss = 0.1400518084615469
 ---- 1 metric test_eval_f1 = 0.42173298726449615
 ---- 1 metric test_eval_time = 5.18602495543672
 ---- 1 metric incorrect_ood_test = 0.0
 ---- 1 metric ood_eval = 0.03598484848484849
 ---- 1 metric ood_eval_time = 6.035037878787879
 ---- 2000 metric q_loss = 0.13656904004514217
 ---- 2 metric test_eval_f1 = 0.46157585889996067
 ---- 2 metric test_eval_time = 6.5310459886841725
 ---- 2 metric incorrect_ood_test = 0.017040998217468805
 ---- 2 metric ood_eval = 0.10037878787878787
 ---- 2 metric ood_eval_time = 11.043560606060606
 ---- 1 metric train_eval_f1 = 0.4143934977332132
 ---- 1 metric train_eval_time = 6.282545998831775
 ---- 1 metric incorrect_ood_train = 0.023458110516934045
 ---- 3000 metric q_loss = 0.13939678977429867
 ---- 3 metric test_eval_f1 = 0.4514036797478512
 ---- 3 metric test_eval_time = 5.591268992082174
 ---- 3 metric incorrect_ood_test = 0.000427807486631016
 ---- 3 metric ood_eval = 0.04924242424242424
 ---- 3 met

KeyboardInterrupt: 

In [12]:
documenter = Documenter(train_dataset= train_dataset,test_dataset= test_dataset, ood_dataset = ood_dataset,full_model= classification_trainer.best["model"], early_model= ddq_model.best["model"],
                        configs= configs
                        )

In [13]:
documenter.document(name= configs["name"])

In [16]:
from sklearn.metrics import f1_score

In [58]:
a = torch.rand(10,4)
print(a, a[-1, : -1])
torch.argmax(a[-1, : -1]).item()

tensor([[0.8008, 0.5225, 0.9946, 0.4995],
        [0.3875, 0.8735, 0.4473, 0.8742],
        [0.2736, 0.7370, 0.9039, 0.4186],
        [0.5684, 0.5814, 0.0498, 0.4500],
        [0.8719, 0.5438, 0.3256, 0.3742],
        [0.3563, 0.9617, 0.8656, 0.3558],
        [0.4970, 0.6278, 0.0561, 0.1948],
        [0.1007, 0.0800, 0.0280, 0.3413],
        [0.8641, 0.9601, 0.7356, 0.5421],
        [0.4536, 0.6649, 0.8181, 0.3578]]) tensor([0.4536, 0.6649, 0.8181])


2

In [29]:
pred,time, labels = EarlyEvaluation(min_steps= 5, device= device, model= ddq_model.best["model"]).predictOnDataset(dataset= test_dataset,enforce_prediction= False)

In [34]:
from sklearn.metrics import f1_score
scores = f1_score(y_true= labels, y_pred= pred, average= None)

0.9167045949507788

In [35]:
scores

array([0.68427095, 0.8716707 , 0.91332712, 0.82959831, 0.91909385,
       0.92327366, 0.61428571, 0.87876844, 0.75036928, 0.72799209,
       0.82420749, 0.32592593, 0.76397516])

In [24]:
test_dataset.label_to_index

{'instagram': 0,
 'hangout': 1,
 'netflix': 2,
 'google-maps': 3,
 'twitter': 4,
 'spotify': 5,
 'facebook': 6,
 'youtube': 7,
 'reddit': 8,
 'pinterest': 9,
 'google-drive': 10,
 'messenger': 11,
 'gmail': 12}

In [15]:
priors = []
for i in range(len(memory)):
    if memory[i].state.label == 3:
        priors.append(p_memory.memory_index_to_priority[i])
priors = np.array(priors)
print(priors.mean())

0.12742096798578842


In [ ]:
batch = p_memory.sample()

In [ ]:
p_memory.tree.total_priority/128

34.95147671434097

In [ ]:
pd.Series(batch["label"]).value_counts()

0    36
2    35
3    33
1    24
Name: count, dtype: int64

In [12]:
EarlyEvaluation(min_steps= 5,device= device,model= ddq_model.best["model"]).getMetrices(dataset= test_dataset,ood_dataset= ood_dataset)

{'micro_f1': 0.8371288555779763,
 'macro_f1': 0.7547126734628223,
 'accuracy': 0.8371288555779763,
 'cm': array([[304,   0,   4,   3,   3,   1,  52,   6,   7,   5,   0,  31,   1],
        [  1, 179,   1,   4,   5,   5,   0,   2,   0,   0,   0,   1,   0],
        [  3,   0, 499,   7,   0,   1,   6,   3,   6,   3,   0,   1,   0],
        [  7,   2,  11, 968,   0,   4,  13,  39,   2,  12,   8,   3,   4],
        [  1,   6,   2,   2, 720,  17,   1,   0,   3,   3,   2,   0,   0],
        [  1,   7,   0,   0,  15, 733,   4,   0,  10,   9,   1,   3,   0],
        [ 18,   1,   1,   5,   3,   1, 276,  10,  10,   6,   2,  27,   2],
        [  4,   0,   0,  15,   5,   0,   2, 739,   1,   0,   2,   0,   1],
        [ 10,   1,   4,   3,  10,   3,  34,   5, 494,  46,   4,  16,   3],
        [ 11,   2,   2,  10,   5,   6,  13,  13,  44, 391,   4,  10,   6],
        [  4,   1,   0,  11,   1,   0,   6,  13,   3,   4, 290,   1,   2],
        [ 14,   0,   3,   1,   0,   0,  45,   3,  15,  10,   0,  74,  

In [ ]:
arr = np.array(list(ddq_model.p_memory.memory_index_to_priority.values()))


In [ ]:
batch = p_memory.sample()

In [ ]:
batch["IS_weights"]

tensor(0.1818)

In [ ]:
p_memory.PER_b_increment_per_sampling

0

In [ ]:
p_memory.PER_b

0.5

In [ ]:
batch["IS_weights"]

torch.Size([128])